In [ ]:
# pip install -r requirements.txt

### Using LangChain basics

You can use prompt templates to insert data into your prompt easily. Don't forget to create environment variables for your OpenAI or Azure OpenAI keys.

In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage
import os
from dotenv import load_dotenv
load_dotenv()
#  https://learn.microsoft.com/en-us/azure/ai-services/openai/reference

True

### Getting Started

In [2]:
# Using completions API
llm = AzureChatOpenAI(deployment_name='gpt-35-turbo',temperature=0)

# invoke llm
llm(
    [
        HumanMessage(
            content='Tell me a joke and output the result in json.'
        )
    ]
)

AIMessage(content='{\n  "joke": "Why don\'t scientists trust atoms? Because they make up everything!"\n}', additional_kwargs={}, example=False)

### Using LangChain to get structured data outputs

You can use LangChain to get JSON responses. Below I am wanting my question and answer in a structured format. But you can build whatever models you want.

In [8]:
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field, validator

In [30]:
# I am chosing Q&A here, but really you can chose any structure you need for your class. 
# It knows how to generate the class structure based on what you put in description - magic?
class QnA(BaseModel):
    question: str = Field(description="question")
    answer: str = Field(description="answer")
    # word: str = Field(description="the first word of the question")

In [31]:
# You can use different parsers, or even construct your own. Pydantic creates nice objects for python so I am using that.
parser = PydanticOutputParser(pydantic_object=QnA)
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [32]:
# Format the prompt
actor_query = "What is a good name for a company that makes colorful socks?"
_input = prompt.format_prompt(query=actor_query)
print(_input)


text='Answer the user query.\nThe output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"question": {"title": "Question", "description": "question", "type": "string"}, "answer": {"title": "Answer", "description": "answer", "type": "string"}}, "required": ["question", "answer"]}\n```\nWhat is a good name for a company that makes colorful socks?\n'


In [33]:
# invoke the llm using the prompt template and formatter
output = llm([HumanMessage(content=_input.to_string())])
print(output.content)

{"question": "What is a good name for a company that makes colorful socks?", "answer": "Rainbow Threads"}


### The following demonstrates how to use Conversation History and Memory in a conversation chain

LangChain can help remember what the user is saying for back and forth chatgpt like capabilities

In [34]:
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


# llm = AzureChatOpenAI(deployment_name='gpt-35-turbo',temperature=0)

# The conversation object manages the back and forward conversation, including memory.
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)

# We just add user input and let the conversation object handle the rest.
conversation.predict(input="What does Microsoft do?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: What does Microsoft do?
AI:

> Finished chain.


'Microsoft is a multinational technology company that develops, manufactures, licenses, supports, and sells computer software, consumer electronics, personal computers, and related services. They are best known for their operating system, Microsoft Windows, which is used by the majority of personal computers worldwide. Microsoft also offers a wide range of other software products, including the Microsoft Office suite, which includes popular applications like Word, Excel, and PowerPoint. Additionally, they provide cloud services through their Azure platform, develop video game consoles like the Xbox, and have a presence in the hardware market with products like the Surface line of tablets and laptops.'

In [35]:
# Adding more user input to the conversation chain keeps the memory, see the verbose output below.
conversation.predict(input="How can they partner together with NASA?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What does Microsoft do?
AI: Microsoft is a multinational technology company that develops, manufactures, licenses, supports, and sells computer software, consumer electronics, personal computers, and related services. They are best known for their operating system, Microsoft Windows, which is used by the majority of personal computers worldwide. Microsoft also offers a wide range of other software products, including the Microsoft Office suite, which includes popular applications like Word, Excel, and PowerPoint. Additionally, they provide cloud services through their Azure platform, develop video game consoles like the Xbox, and have a presence in the hardware 

"Microsoft has a long-standing partnership with NASA. They collaborate on various projects and initiatives that involve the use of technology in space exploration and research. One notable example is the use of Microsoft's HoloLens mixed reality headset in NASA's Project Sidekick, which aims to provide astronauts with virtual assistance during space missions. Microsoft also works with NASA on data analysis and visualization tools, as well as cloud computing solutions to support the agency's scientific research and data processing needs. Additionally, Microsoft has partnered with NASA to develop educational programs and initiatives to inspire and engage students in STEM fields."

### LangChain Tools

You can set custom tools to be used, for example here is the bing search tool

In [7]:
import os
os.environ["BING_SUBSCRIPTION_KEY"] = ""
os.environ["BING_SEARCH_URL"] = "https://api.bing.microsoft.com/v7.0/search"

from langchain.utilities import BingSearchAPIWrapper
search = BingSearchAPIWrapper(k=1)
search.run("Dave Enright from Singapore on LinkedIn")

'About. I help companies transform into data driven organisations so they can become predictive instead of reactive. I’ve created a ‘data adoption framework’ that brings organisation from strategy...'

### You can use agents to orchestrate multiple tools together

Below the agent is being told to use the Bing Search tool when it needs to answer current events.The agent will use this tool based on the kinds of questions the user is asking.

In [17]:
from langchain.agents import Tool, AgentExecutor, BaseSingleActionAgent
from langchain.utilities import BingSearchAPIWrapper

# The tool description helps langchain work out when to use the tool. Be careful with your descriptions as its not completely deterministic.
search = BingSearchAPIWrapper(k=1)
tools = [
    Tool(
        name = "Intermediate Answer",
        func=search.run,
        description="Use this when you need up to date information that is timely",
        return_direct=True
    )
]

Initialising the agent to use the tool

In [20]:
from typing import List, Tuple, Any, Union
from langchain.agents import initialize_agent
from langchain.agents import AgentType

# There are different kinds of agents documented on the langchain website. This particular one is basically just a chat with search agent.
self_ask_with_search = initialize_agent(tools, llm, agent=AgentType.SELF_ASK_WITH_SEARCH, verbose=True)
self_ask_with_search.run("Give me a list of to places to visit where weather is not too hot and do not have a lot of tourists")



> Entering new AgentExecutor chain...
 Yes.
Follow up: What type of places are you looking for?
Intermediate answer: You can search for areas of interest, things to do, or notable locations in Google Maps. Find places like nearby museums, new restaurants, and popular bars and clubs. You can also find ratings and...


> Finished chain.


'You can search for areas of interest, things to do, or notable locations in Google Maps. Find places like nearby museums, new restaurants, and popular bars and clubs. You can also find ratings and...'

In [8]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.llms import OpenAI

llm = OpenAI(temperature=0)
tools = load_tools(["bing-search", "llm-math"], llm=llm)

agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("Who is Leo DiCaprio's girlfriend? What is her current age raised to the 0.43 power?")



> Entering new AgentExecutor chain...
 I need to find out who Leo DiCaprio's girlfriend is and then calculate her age raised to the 0.43 power.
Action: Bing Search
Action Input: "Leo DiCaprio girlfriend"
Observation: <b>Leonardo DiCaprio</b> &amp; Camila Morrone&#39;s Full Relationship Timeline Celebrity Celebrity News Everything You Need to Know About <b>Leonardo DiCaprio</b> and Camila Morrone&#39;s Relationship From the beginning... <b>Leonardo</b> <b>DiCaprio</b> started dating hot 25 year olds back in the roaring &#39;90s and...truly nothing has changed! Other than the people he&#39;s dating, of course, because lord knows this man won&#39;t stay... <b>Leonardo</b> <b>DiCaprio</b> recently split with his <b>girlfriend</b> of several years, Camila Morrone. In the past, the &quot;Titanic&quot; actor has been romantically linked to Gisele Bündchen, Rihanna, and more. <b>DiCaprio</b>&#39;s tendency to date much younger women has spawned countless memes and jokes online. <b>Leonardo</

"Camila Morrone is Leo DiCaprio's girlfriend and her current age raised to the 0.43 power is 3.991298452658078."

### Work with Documents

In [36]:
from langchain.document_loaders import PyPDFLoader

# There are different document loaders available, this is the basic PDF loader.
loader = PyPDFLoader("Benefit_Options.pdf")
pages = loader.load_and_split()

In [37]:
# Splitting into pages is useful to give page number back as a source
pages[0]

Document(page_content='Contoso Electronics  \nPlan and Benefit Packages', metadata={'source': 'Benefit_Options.pdf', 'page': 0})

In [38]:
from langchain.vectorstores import FAISS
from langchain.embeddings.openai import OpenAIEmbeddings

# This is creating an index based on embeddings, but as you can see by the output it's a bit ugly to work with.

faiss_index = FAISS.from_documents(pages, OpenAIEmbeddings(chunk_size=1))
docs = faiss_index.similarity_search("What's the difference between plus and standard?", k=2)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300])

3: offers a wider range of prescription drug coverage than Northwind Standard. Both plans offer coverage 
for vision and dental services, as well as medical services.  
Next Steps  
We hope that this information has been helpful in understanding the differe nces between Northwind 
Health Plus and North
2: Welcome to Contoso  Electronics ! We are excited to offer our employees two comprehensive health 
insurance plans through Northwind Health.  
Northwind Health Plus  
Northwind Health Plus is a comprehensive plan that provides comprehensive coverage for medical, 
vision, and dental services. T his pl


In [11]:
# Does not work yet with Azure.
#from langchain.indexes import VectorstoreIndexCreator

# Using the vector store from the loader means langchain can abstract away a lot of the under the hood stuff.
# Note I am using a local python package for the vectorDB. You may get errors.
#index = VectorstoreIndexCreator().from_loaders([loader])

In [39]:
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

chain = load_qa_with_sources_chain(llm, chain_type="stuff")

query = "What's the difference between plus and standard?"
chain({"input_documents": docs, "question": query}, return_only_outputs=True)

{'output_text': 'The main difference between Northwind Health Plus and Northwind Standard is that Northwind Health Plus offers more comprehensive coverage. Northwind Health Plus provides coverage for emergency services, mental health and substance abuse, and out-of-network services, while Northwind Standard does not. Additionally, Northwind Health Plus offers a wider range of prescription drug coverage, including generic, brand-name, and specialty drugs, whereas Northwind Standard only covers generic and brand-name drugs. Both plans offer coverage for vision and dental services, as well as medical services. \nSOURCES: Benefit_Options.pdf'}

In [40]:
# Langchain CSV example

from langchain.agents import create_csv_agent
from langchain.llms import AzureOpenAI

agent = create_csv_agent(llm, 'sample.csv', verbose=True)

In [41]:
agent.run("how many rows are there?")



> Entering new AgentExecutor chain...
Thought: To find the number of rows in the dataframe, I can use the `shape` attribute of the dataframe.

Action: python_repl_ast
Action Input: df.shape[0]
Observation: 5
Thought:The dataframe has 5 rows.
Final Answer: 5

> Finished chain.


'5'

In [42]:
from langchain.llms import AzureOpenAI
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter

# llm = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

text_splitter = CharacterTextSplitter()

with open("state_of_the_union.txt", encoding="utf8") as f:
    sotu = f.read()
texts = text_splitter.split_text(sotu)

In [48]:
from langchain.docstore.document import Document
docs = [Document(page_content=t) for t in texts[:3]]

In [50]:
from langchain.chains.summarize import load_summarize_chain
chain = load_summarize_chain(llm, chain_type="map_reduce")
chain.run(docs)

'President Biden addresses the conflict between Russia and Ukraine in his State of the Union address, emphasizing the unity and resolve of the United States and its allies in holding Russia accountable. The US is taking action against Russian oligarchs and corrupt leaders, implementing economic sanctions, and providing support to Ukraine. American forces are not engaged in the conflict but are prepared to defend NATO allies. The US is implementing targeted sanctions on the Russian economy and releasing oil reserves to help with gas prices. The speaker highlights the rise of democracies and the support for Ukraine worldwide. The American Rescue Plan is praised for providing relief and support, contrasting with previous tax cuts. The importance of investing in infrastructure is emphasized, celebrating the passage of a bipartisan infrastructure law.'

In [51]:
from langchain.chains.question_answering import load_qa_chain
chain = load_qa_chain(llm, chain_type="stuff")
query = "Did the president say selamat pagi in english?"
chain.run(input_documents=docs, question=query)

'No, the President did not say "selamat pagi" in English.'

### Azure Cognitive Search

In [ ]:
# test using cog search vector store.
# reference: https://python.langchain.com/docs/integrations/vectorstores/azuresearch
# !pip install --index-url=https://pkgs.dev.azure.com/azure-sdk/public/_packaging/azure-sdk-for-python/pypi/simple/ azure-search-documents==11.4.0a20230509004
# !pip install azure-identity
# !pip install azure-search-documents==11.4.0b6
# !pip install langchain --upgrade

In [3]:
import os, json
import openai
from dotenv import load_dotenv
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.schema import BaseRetriever
from langchain.vectorstores.azuresearch import AzureSearch

In [4]:
vector_store_address: str = 'https://globecogse.search.windows.net'
vector_store_password: str = 'Jk3aMWZNUeOdjiPZxDWgaWS4lXOY001YQcRTZXVUbZAzSeCiFO0l'
index_name: str = 'globevectorindex'

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(chunk_size=1)  
vector_store: AzureSearch = AzureSearch(azure_search_endpoint=vector_store_address,  
                                        azure_search_key=vector_store_password,  
                                        index_name=index_name,  
                                        embedding_function=embeddings.embed_query) 


In [5]:
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
loader = TextLoader('state_of_the_union.txt', encoding='utf-8')
documents = loader.load()

In [6]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)

Created a chunk of size 1434, which is longer than the specified 1000
Created a chunk of size 3063, which is longer than the specified 1000


In [ ]:
vector_store.add_documents(documents=docs)

In [9]:
# using retrievalQA
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_llm(llm=llm, retriever=vector_store.as_retriever(), verbose=False)
qa.run('What did the president say about Ketanji Brown Jackson')

'The president said that he has nominated Ketanji Brown Jackson to serve on the United States Supreme Court.'

In [10]:
# using similarity search
docus = vector_store.similarity_search(query="What did the president say about Ketanji Brown Jackson", k=3, search_type='similarity')
print(docus[0].page_content)

And we all know—no matter what your ideology, we all know one of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. As I did 4 days ago, I've nominated a Circuit Court of Appeals—Ketanji Brown Jackson.


### VectoreStoreIndex (Not Working)

In [11]:
# use custom index with existing vectorstore. This opens up the ability to do query_with_sources etc
from langchain.indexes import VectorstoreIndexCreator

vector_store: AzureSearch = AzureSearch(azure_search_endpoint=vector_store_address,  
                                        azure_search_key=vector_store_password,  
                                        index_name=index_name,  
                                        embedding_function=embeddings.embed_query) 

index_wrapper = VectorstoreIndexCreator(
    vectorstore_cls=AzureSearch,
    embedding=OpenAIEmbeddings(chunk_size=1),
    text_splitter=CharacterTextSplitter(chunk_size=1024, chunk_overlap=128)
)

In [15]:
# query = "What did the president say about Ketanji Brown Jackson"
# index_wrapper.query_with_sources(query)

AttributeError: 'VectorstoreIndexCreator' object has no attribute 'query_with_sources'

### Google Drive Loader

In [43]:
from langchain.document_loaders import GoogleDriveLoader

In [44]:
loader = GoogleDriveLoader(document_ids=["1s_-umZEhfcNVcyEZ6FbOi0pPIr-UEadEbz_OyAerpqc"])
docs = loader.load()
len(docs)

1

In [45]:
len(docs[0].page_content)

6042

In [46]:
# document question and aswering
from langchain.chains.question_answering import load_qa_chain
query = 'What are the tracks offered by the cadetship program?'
chain = load_qa_chain(llm, chain_type='stuff', verbose=False)
chain.run(input_documents=docs,question=query)

'The tracks offered by the cadetship program are AI/ML, Software Development, Product Management and Delivery, Security, IT Quality Management and Testing, and Platform Management.'

In [47]:
# text splitting 
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs_split = text_splitter.split_documents(docs)

len(docs_split)
len(docs_split[0].page_content)

992

In [48]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch

vector_store_address: str = 'https://globecogse.search.windows.net'
vector_store_password: str = 'Jk3aMWZNUeOdjiPZxDWgaWS4lXOY001YQcRTZXVUbZAzSeCiFO0l'
index_name: str = 'globevectorindex'

embeddings: OpenAIEmbeddings = OpenAIEmbeddings(chunk_size=1)  
vector_store: AzureSearch = AzureSearch(azure_search_endpoint=vector_store_address,  
                                        azure_search_key=vector_store_password,  
                                        index_name=index_name,  
                                        embedding_function=embeddings.embed_query) 
                                        
vector_store.add_documents(documents=docs_split)

['MWM5Zjc3MDAtZjVkZS00YjRiLWEwYTYtZTJkMGY2OTE5NDgy',
 'Y2U0ZWNiOTEtNzBmZC00MzQzLWExYjEtYzQzYzlmYWUwOGVh',
 'YTlhYTdkNTYtNzdiZC00MWIzLTkwMDUtNTYzYzdiMDc4ZTUy',
 'YjE2ODY3ZWMtMDhmYi00ZGZiLThjMjMtYzMzNTg5YjVmOTRj',
 'NTkxNzA4ZTYtMWU5OC00MDQ3LTg5NzMtZmRjNTU2M2QxYTAw',
 'MGUxNzBmOGMtYTI4Ny00OGQ3LWEyNjktMWIzYjE0ZWMwNzdk',
 'MWFmYTFlNDQtZjVlNS00ZWFhLWIxMzQtMzJlYTUwNDZiN2Jh']

In [49]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_llm(llm=llm, retriever=vector_store.as_retriever(), verbose=False)
qa.run('What are the tracks offered by the cadetship program?')

'The tracks offered by the cadetship program are AI/ML, Software Development, Product Management and Delivery, Security, IT Quality Management and Testing, and Platform Management.'

In [51]:
from langchain.chains import RetrievalQA
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())
qa.run('What are the tracks offered by the cadetship program?')

'The tracks offered by the cadetship program are AI/ML, Software Development, Product Management and Delivery, Security, IT Quality Management and Testing, and Platform Management.'